In [53]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/toxicity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/identity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv


In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
import multiprocessing
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re
import string



In [55]:
files=['../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv',
       '../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv',
      '/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv']

def load_data(file):
    return pd.read_csv(file)
with multiprocessing.Pool() as pool:
    test,train,sub=pool.map(load_data,files)

In [56]:
train['target'].nunique()

2913

In [57]:
print(train.shape)
print(test.shape)

(1804874, 45)
(97320, 2)


In [58]:
train.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count'],
      dtype='object')

In [59]:
test.columns

Index(['id', 'comment_text'], dtype='object')

In [60]:
train_data=train[['comment_text','target']]

In [62]:
train_data.head()

,comment_text,target
0,"This is so cool. It's like, 'would you want yo...",0.000000
1,Thank you!! This would make my life a lot less...,0.000000
2,This is such an urgent design problem; kudos t...,0.000000
3,Is this something I'll be able to install on m...,0.000000
4,haha you guys are a bunch of losers.,0.893617


In [63]:
del train

In [64]:
train_data_filtered_toxic = train_data[train_data['target'] >= 0.8]

In [65]:
train_data_filtered_toxic

,comment_text,target
4,haha you guys are a bunch of losers.,0.893617
31,Yet call out all Muslims for the acts of a few...,0.912500
34,This bitch is nuts. Who would read a book by a...,0.830769
289,You're an idiot.,0.968750
306,Who cares!? Stark trek and Star Wars fans are ...,0.900000
...,...,...
1804600,"Wow. What entitled, arrogant , immature, loser...",0.805556
1804712,Human scum.,0.862745
1804713,This dirtbag is no Marine; he's a sadistic ter...,0.800000
1804753,I saved where I could because my children matt...,0.833333


In [66]:
train_data_filtered_nottoxic=train_data[train_data['target'] <= 0.2]

In [67]:
train_data_filtered_nottoxic

,comment_text,target
0,"This is so cool. It's like, 'would you want yo...",0.0
1,Thank you!! This would make my life a lot less...,0.0
2,This is such an urgent design problem; kudos t...,0.0
3,Is this something I'll be able to install on m...,0.0
7,FFFFUUUUUUUUUUUUUUU,0.0
...,...,...
1804868,"cont....GBA: ""here's the summation of that ""l...",0.0
1804869,"Maybe the tax on ""things"" would be collected w...",0.0
1804870,What do you call people who STILL think the di...,0.0
1804871,"thank you ,,,right or wrong,,, i am following ...",0.0


In [68]:
# remove all numbers with letters attached to them
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)

# '[%s]' % re.escape(string.punctuation),' ' - replace punctuation with white space
# .lower() - convert all strings to lowercase 
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

# Remove all '\n' in the string and replace it with a space
remove_n = lambda x: re.sub("\n", " ", x)

# Remove all non-ascii characters 
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ', x)

# Apply all the lambda functions wrote previously through .map on the comments column
train_data_filtered_nottoxic['comment_text'] = train_data_filtered_nottoxic['comment_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
train_data_filtered_toxic['comment_text'] = train_data_filtered_toxic['comment_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
test['comment_text'] = test['comment_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)

In [69]:
train_data_filtered_nottoxic

,comment_text,target
0,this is so cool it s like would you want yo...,0.0
1,thank you this would make my life a lot less...,0.0
2,this is such an urgent design problem kudos t...,0.0
3,is this something i ll be able to install on m...,0.0
7,ffffuuuuuuuuuuuuuuu,0.0
...,...,...
1804868,cont gba here s the summation of that l...,0.0
1804869,maybe the tax on things would be collected w...,0.0
1804870,what do you call people who still think the di...,0.0
1804871,thank you right or wrong i am following ...,0.0


In [70]:
train_data_filtered_toxic

,comment_text,target
4,haha you guys are a bunch of losers,0.893617
31,yet call out all muslims for the acts of a few...,0.912500
34,this bitch is nuts who would read a book by a...,0.830769
289,you re an idiot,0.968750
306,who cares stark trek and star wars fans are ...,0.900000
...,...,...
1804600,wow what entitled arrogant immature loser...,0.805556
1804712,human scum,0.862745
1804713,this dirtbag is no marine he s a sadistic ter...,0.800000
1804753,i saved where i could because my children matt...,0.833333


In [71]:
train_min=pd.concat([train_data_filtered_toxic,train_data_filtered_nottoxic],axis=0)
train_min.shape

(1565511, 2)

In [72]:
del train_data_filtered_toxic
del train_data_filtered_nottoxic


In [75]:
tfidfvec=TfidfVectorizer(ngram_range=(1,1))
train_min_vec=tfidfvec.fit_transform(train_min['comment_text'])
    

In [76]:
train_min_vec.shape

(1565511, 251506)

In [77]:
test.head()

,id,comment_text
0,7097320,integrity means that you pay your debts d...
1,7097321,this is malfeasance by the administrator and t...
2,7097322,spoken like a true elitist but look out ...
3,7097323,paul thank you for your kind words i do in...
4,7097324,sorry you missed high school eisenhower sent ...


In [78]:
test_vec=tfidfvec.transform(test['comment_text'])

In [79]:
test_vec

<97320x251506 sparse matrix of type '<class 'numpy.float64'>'
	with 3694933 stored elements in Compressed Sparse Row format>

In [80]:
X=train_min_vec

In [81]:
y=np.where(train_min['target']>0.8,1.0,0.0)

In [82]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [83]:
def ModelLG(x_train, x_test, y_train, y_test):
    LG = LogisticRegression()
    LG.fit(x_train, y_train)
    pred=LG.predict(x_test)
    score=f1_score(pred,y_test)
    return score

In [84]:
Score=ModelLG(x_train, x_test, y_train, y_test)

In [85]:
Score

0.6505526102974212

In [86]:
LG = LogisticRegression()
LG.fit(X, y)

LogisticRegression()

In [87]:
pred=LG.predict_proba(test_vec)

In [96]:
sub['prediction'] = LG.predict_proba(test_vec)[:, 1]
sub.to_csv('submission.csv', index=False)

In [97]:
sub

,id,prediction
0,7097320,0.016777
1,7097321,0.007665
2,7097322,0.026643
3,7097323,0.001734
4,7097324,0.002385
...,...,...
97315,7194635,0.003690
97316,7194636,0.003702
97317,7194637,0.030323
97318,7194638,0.013585
